In [1]:
import numpy as np
import pandas as pd
import re
import fuzzywuzzy
from fuzzywuzzy import process

C:\Users\User\Anaconda3\lib\site-packages\fuzzywuzzy\fuzz.py:35: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [3]:
# Carrega o dataset de veículos
df_veiculos = pd.read_csv('veiculos.csv', encoding='ISO-8859-1')

# Separa apenas os campos de interesse
df_veiculos = df_veiculos[['CODIGO', 'PLACA', 'MODELO', 'MARCA']]

# Com base em um banco de dados já correto de marcas
df_marcas_ok = pd.read_csv(r'marcas.csv', encoding='latin1')

df_veiculos.shape


(898, 4)

In [4]:
df_veiculos.head()

,CODIGO,PLACA,MODELO,MARCA
0,1,YNU-2233,NaN,NaN
1,2,NYH-0213,CAMINHAO VOLKSWAGEN 24.280,VOLKSWAGEN 24280
2,4,VWU-3245,CAMINHÃO VOLKSWAGENS 24.250,VOLKSWAGEN 24.250
3,5,UXB-7609,CAMINHÃO VOLKSWAGEN 24.250,VOLKSWAGEN 24.250
4,6,IGM-1705,CAMINHÃO VOLKSWAGEN 24.250,VOLKSWAGEN 24.250


In [5]:
df_marcas_ok.head()

,NOME
0,AGRALE
1,AUDI
2,BMW
3,FIAT
4,FORD


In [6]:
# Verifica quantos veículos possuem a marca com problemas de grafia

df_veiculos = df_veiculos.merge(df_marcas_ok, how='left', 
                                left_on='MARCA', right_on='NOME')

df_veiculos.rename(columns={'NOME': 'PROBLEMA_MARCA'}, inplace=True)

porc_problema = df_veiculos[df_veiculos['PROBLEMA_MARCA'].isnull()].shape[0] / df_veiculos.shape[0] 
print('Total de veículos com problema na marca:', porc_problema*100, '%')

Total de veículos com problema na marca: 48.91304347826087 %


In [7]:
# visualiza quantas marcas existem
marcas_inicio = df_veiculos['MARCA'].unique().shape[0]
print(marcas_inicio)
print(df_veiculos['MARCA'].unique())


214
[nan 'VOLKSWAGEN 24280' 'VOLKSWAGEN 24.250' 'VOLKSWAGEN'
 'VOLKSWAGEN  24.280' 'SR GUERRA' 'SCANIA' 'SCANIA RANDON SRCA'
 'SCANIA R440' 'SCANIA P 340' 'GUERRA AG GR' 'MERCEDES BENZ'
 'LOIBRELATTO SRCS 3E' 'SCANIA R124' 'GUERRA' 'SCANIA G400' 'FACCHINI'
 'SCANIA G380' 'RANDON' 'RANDON SR' 'SCANIA FH12' 'LIBRELATTO' 'FORD CARGO'
 'SR LIBRELATTO' 'VOLVO' 'FORD' 'RANDON SR TR' 'SCANIA R113'
 'MERCEDES BENZ ATEGO' 'FH 420' 'VOLVO FH' 'FACHINI' 'VOLKSWAGEN MAN'
 'JARDINOX' 'GUERRA CHARGER' 'RECRUSUL' 'SCANIA T113' 'ATEGO 2426'
 'PORTA CONTAINER' 'VW 24.280' 'MERCEDES ATEGO 2429' 'SCANIA T112'
 'MERCEDES LS 1935' 'SCANIA P114' 'IVECO STRALYS' 'KRONE PRANCHA'
 'VW 24.390' 'IVECO' 'FORD CARGA 2428' 'MAN 29.440' 'TRIEL HT'
 'MERCEDES AXOR' 'FACCHINI SYDER' 'RODOLINEA' 'SCANIA T114' 'SR RANDON'
 'MERCEDES' 'VOLKS MAN' 'VOLVO NL10' 'SCANIA P 114' 'FIBRASIL'
 'SCANIA T124' 'NOMA' 'SOUFER' 'VW 19.320' 'SCHIFFER' 'SEMI REBOQUE'
 'STRALYS' 'SR/GUERRA' 'MAN 28.440' 'RANSON' 'VW 23220' 'SR LIBRELATO

In [8]:
# cria um atributo marca_antes para futura comparação
df_veiculos['MARCA_ANTES'] = df_veiculos['MARCA']

# remove números e caracteres estranhos para simplificar as marcas
df_veiculos['MARCA'] = df_veiculos['MARCA'].str.replace(r'[^A-Za-z]+', ' ')

# transforma NaN em valores em branco
df_veiculos['MARCA'].replace(np.nan, '', inplace=True)

# adiciona espaços em branco para ajudar a remover letras isoladas
df_veiculos['MARCA'] = ' '+df_veiculos['MARCA']+' '


In [9]:
# Já com essas ações é possível visualizar uma redução de marcas
print(df_veiculos['MARCA'].unique().shape[0])

157


In [10]:
# Função auxiliar para remover palavras por espaços em branco
def replacevalues(df, column, search, replace):
    for item in search:
        r = re.compile(' '+item+' ', flags=re.IGNORECASE)
        df[column] = df[column].str.replace(r, replace)


In [11]:
# Em uma rápida inspeção é possível verificar que poucos veículos
# possuem marcas com menos de 3 caracteres, exceto alguns:
dont_remove = ['MAN', 'DAF', 'MWM', 'VW', 'M']

# Para o restante das marcas, cria uma lista para remoção
remove = [x.split() for x in df_veiculos['MARCA'].unique()]
remove = [item for sublist in remove for item in sublist]
remove = [item for item in remove if len(item) <= 3 and item not in dont_remove]


In [12]:
print('Antes:', len(df_veiculos['MARCA'].unique()))
# remove todas as palavras na lista de remoção
replacevalues(df_veiculos, 'MARCA', remove, ' ')

# cria uma outra lista, com palavras com mais de 3 caracteres
# e que não são marcas
remove = ['SRCA', 'SRCS', 'PRANCHA', 'SRFG', 'STRALIS', 'CARGA', 'CARGO',
          'AXOR', 'ATEGO', 'ACTROS', 'ABERT', 'CHARGER', 'STRALYS', 'SYDER',
          'PORTA CONTAINER', 'CAVALO', 'SEMI REBOQUE', 'TRUCK', 'REBOQUE',
          'TRAC TRATOR']
replacevalues(df_veiculos, 'MARCA', remove, ' ')

# remove os espaços em branco extras
df_veiculos['MARCA'] = df_veiculos['MARCA'].str.strip()

print('Depois:', len(df_veiculos['MARCA'].unique()))


Antes: 157
Depois: 57


In [12]:
df_veiculos['MARCA'].unique()


array(['  ', ' VOLKSWAGEN  ', ' VOLKSWAGEN ', ' SR GUERRA ', ' SCANIA ',
       ' SCANIA RANDON SRCA ', ' SCANIA R  ', ' SCANIA P  ',
       ' GUERRA AG GR ', ' MERCEDES BENZ ', ' LOIBRELATTO SRCS E ',
       ' GUERRA ', ' SCANIA G  ', ' FACCHINI ', ' RANDON ', ' RANDON SR ',
       ' SCANIA FH  ', ' LIBRELATTO ', ' FORD CARGO ', ' SR LIBRELATTO ',
       ' VOLVO ', ' FORD ', ' RANDON SR TR ', ' MERCEDES BENZ ATEGO ',
       ' FH  ', ' VOLVO FH ', ' FACHINI ', ' VOLKSWAGEN MAN ',
       ' JARDINOX ', ' GUERRA CHARGER ', ' RECRUSUL ', ' SCANIA T  ',
       ' ATEGO  ', ' PORTA CONTAINER ', ' VW  ', ' MERCEDES ATEGO  ',
       ' MERCEDES LS  ', ' IVECO STRALYS ', ' KRONE PRANCHA ', ' IVECO ',
       ' FORD CARGA  ', ' MAN  ', ' TRIEL HT ', ' MERCEDES AXOR ',
       ' FACCHINI SYDER ', ' RODOLINEA ', ' SR RANDON ', ' MERCEDES ',
       ' VOLKS MAN ', ' VOLVO NL  ', ' FIBRASIL ', ' NOMA ', ' SOUFER ',
       ' SCHIFFER ', ' SEMI REBOQUE ', ' STRALYS ', ' RANSON ',
       ' SR LIBRELATO ', '

In [14]:
# Para algumas marcas, é necessário alterar "manualmente",
# pois há abreviações e outros erros
print('Antes:', len(df_veiculos['MARCA'].unique()))
df_veiculos['MARCA'].replace({'VOLKSWAGEN MAN': 'VOLKSWAGEN',
                              'VOLKS MAN': 'VOLKSWAGEN',
                              'VOLKS': 'VOLKSWAGEN',
                              'VW MWM': 'VOLKSWAGEN',
                              'VW': 'VOLKSWAGEN',
                              'M BENZ': 'MERCEDES-BENZ',
                              'MERCEDES': 'MERCEDES-BENZ',
                              'SCANIA RANDON': 'SCANIA',
                              'NOPIA': 'NOMA'},
                             inplace=True)
print('Depois:', len(df_veiculos['MARCA'].unique()))


Antes: 57
Depois: 49


In [15]:
# Verifica novamente as marcas, ainda há alguns erros de grafia
# que serão corrigidos em seguida
df_veiculos['MARCA'].unique()


array(['', 'VOLKSWAGEN', 'GUERRA', 'SCANIA', 'MERCEDES BENZ',
       'LOIBRELATTO', 'FACCHINI', 'RANDON', 'LIBRELATTO', 'FORD', 'VOLVO',
       'FACHINI', 'JARDINOX', 'RECRUSUL', 'MERCEDES-BENZ', 'IVECO',
       'KRONE', 'MAN', 'TRIEL', 'RODOLINEA', 'FIBRASIL', 'NOMA', 'SOUFER',
       'SCHIFFER', 'RANSON', 'LIBRELATO', 'MARINO', 'GURRA', 'MANN',
       'MERCEDZ BENS', 'FACCHINI TANQUE', 'FACCIHINI', 'RANDONSP', 'GUERA',
       'ELLFEN', 'LIBERATO', 'DAF', 'VOVLO', 'IDAF', 'MERCEDES BENZA',
       'RANDN', 'FACCHIN', 'VOLKSWAGEM', 'RAMDON', 'RODOTECNICA', 'FORS',
       'THERMOSUL', 'NOMAQ', 'FACHIN'], dtype=object)

In [16]:
# Esta função utiliza a biblioteca fuzzywuzzy para comparar cada marca com
# uma String que está correta, caso há algum problema de grafia a marca
# é substituída pela forma correta.

def replace_matches_in_column(df, column, string_to_match, min_ratio=90, only_test=False):
    # function to replace rows in the provided column of the provided dataframe
    # that match the provided string above the provided ratio with
    # the provided string

    # get a list of unique strings
    strings = df[column].unique()

    # get the top 10 closest matches to our input string
    matches = fuzzywuzzy.process.extract(string_to_match, strings,
                                         limit=10,
                                         scorer=fuzzywuzzy.fuzz.token_sort_ratio)

    # only get matches with a ratio > x
    close_matches = [matches[0] for matches in matches if matches[1] >= min_ratio]
    if len(close_matches) > 0:
        print(string_to_match, ':', close_matches)

    if not(only_test):
        # get the rows of all the close matches in our dataframe
        rows_with_matches = df[column].isin(close_matches)

        # replace all rows with close matches with the input matches
        df.loc[rows_with_matches, column] = string_to_match



In [17]:
print('Antes:', len(df_veiculos['MARCA'].unique()))

# procuro uniformizar a grafia de cada marca corretamente
for marca in df_marcas_ok['NOME']:
    replace_matches_in_column(df=df_veiculos, column='MARCA', 
                              string_to_match=marca, min_ratio=70)

print('Depois:', len(df_veiculos['MARCA'].unique()))

Antes: 49
FORD : ['FORD', 'FORS']
IVECO : ['IVECO']
MERCEDES-BENZ : ['MERCEDES BENZ', 'MERCEDES-BENZ', 'MERCEDES BENZA', 'MERCEDZ BENS']
VOLKSWAGEN : ['VOLKSWAGEN', 'VOLKSWAGEM']
SCANIA : ['SCANIA']
VOLVO : ['VOLVO', 'VOVLO']
MAN : ['MAN', 'MANN']
FACCHINI : ['FACCHINI', 'FACCIHINI', 'FACHINI', 'FACCHIN', 'FACHIN', 'FACCHINI TANQUE']
NOMA : ['NOMA', 'NOMAQ']
RODOTÉCNICA : ['RODOTECNICA']
RANDON : ['RANDON', 'RANDN', 'RANDONSP', 'RANSON', 'RAMDON']
LIBRELATTO : ['LIBRELATTO', 'LOIBRELATTO', 'LIBRELATO', 'LIBERATO']
JARDINOX : ['JARDINOX', 'MARINO']
GUERRA : ['GUERRA', 'GURRA', 'GUERA']
KRONE : ['KRONE']
MAN : ['MAN']
NOMA : ['NOMA']
SOUFER : ['SOUFER']
DAF : ['DAF', 'IDAF']
RODOLINEA : ['RODOLINEA']
FIBRASIL : ['FIBRASIL']
THERMOSUL : ['THERMOSUL']
RECRUSUL : ['RECRUSUL']
TRIEL : ['TRIEL']
SCHIFFER : ['SCHIFFER']
ELLFEN : ['ELLFEN']
Depois: 25


In [18]:
# Na visualização abaixo é possível perceber que ainda há alguns veículos
# que não foi possível definir a marca
df_veiculos[df_veiculos['MARCA'] == ''][['PLACA', 'MODELO', 'MARCA']]


,PLACA,MODELO,MARCA
0,YNU-2233,NaN,
48,ETH-6614,VOLVO FH 420,
61,UPN-0373,MERCEDES BENZ ATEGO,
63,GXM-3837,SEMI REBOQUE GUERRA,
150,FQW-9890,SEMI REBOQUE SR,
158,JHB-3302,CAR/TRATOR IVECO STRALYS,
196,JLY-0666,CARGO 2423,
202,PEZ-4311,MERCEDESA BENZ ATEGO,
205,VNT-7308,MERCEDES BENZ AXOR 2540,
208,JWZ-8988,SR LIBRELATTO,


In [19]:
def set_marcas(codigos, marca):
    rows = df_veiculos['CODIGO'].isin(codigos)
    df_veiculos.loc[rows, 'MARCA'] = marca

In [20]:
# Aqui necessita um trabalho um pouco manual,
# onde com base na informação do modelo facilmente 
# se encontra na internet a marca do veículo.
# Depois com a ajuda da função set_marcas é possível 
# atribuir manualmente a marca a estes veículos

set_marcas(codigos=[57, 392], marca='VOLVO')
set_marcas(codigos=[544, 609, 696], marca='VOLKSWAGEN')
set_marcas(codigos=[70, 207, 210, 880], marca='MERCEDES-BENZ')
set_marcas(codigos=[165], marca='IVECO')
set_marcas(codigos=[202, 282], marca='FORD')
set_marcas(codigos=[792], marca='SCANIA')
set_marcas(codigos=[72], marca='GUERRA')
set_marcas(codigos=[213], marca='LIBRELATTO')

# quando não há informação do modelo nem da marca
# atribui-se o valor de Não Especificado
nao_especificado = df_veiculos[df_veiculos['MARCA'] == '']['CODIGO']
set_marcas(codigos=nao_especificado, marca='NAO ESPEC.')


In [21]:
marcas_final = len(df_veiculos['MARCA'].unique())
print('Total de marcas:', marcas_final)
print('Redução de grafias diferentes: ', (1-(marcas_final/marcas_inicio))*100, '%')


Total de marcas: 25
Redução de grafias diferentes:  88.3177570093458 %


In [22]:
df_veiculos['MARCA'].unique()


array(['NAO ESPEC.', 'VOLKSWAGEN', 'GUERRA', 'SCANIA', 'MERCEDES-BENZ',
       'LIBRELATTO', 'FACCHINI', 'RANDON', 'FORD', 'VOLVO', 'JARDINOX',
       'RECRUSUL', 'IVECO', 'KRONE', 'MAN', 'TRIEL', 'RODOLINEA',
       'FIBRASIL', 'NOMA', 'SOUFER', 'SCHIFFER', 'ELLFEN', 'DAF',
       'RODOTÉCNICA', 'THERMOSUL'], dtype=object)

In [23]:
df_veiculos = df_veiculos.merge(df_marcas_ok, how='left', 
                                left_on='MARCA', right_on='NOME')

df_veiculos.rename(columns={'NOME': 'PROBLEMA_MARCA_FINAL'}, inplace=True)

porc_problema = df_veiculos[df_veiculos['PROBLEMA_MARCA_FINAL'].isnull()].shape[0] / df_veiculos.shape[0] 
print('Total de veículos com problema na marca (final):', porc_problema*100, '%')


Total de veículos com problema na marca (final): 6.578947368421052 %


In [24]:
df_veiculos[df_veiculos['PROBLEMA_MARCA_FINAL'].isnull()]

,CODIGO,PLACA,MODELO,MARCA,PROBLEMA_MARCA,MARCA_ANTES,PROBLEMA_MARCA_FINAL
0,1,YNU-2233,NaN,NAO ESPEC.,NaN,NaN,NaN
153,158,FQW-9890,SEMI REBOQUE SR,NAO ESPEC.,NaN,SEMI REBOQUE,NaN
342,301,ZDT-5217,NaN,NAO ESPEC.,NaN,NaN,NaN
343,302,QIV-8738,NaN,NAO ESPEC.,NaN,NaN,NaN
344,303,GXD-8431,NaN,NAO ESPEC.,NaN,NaN,NaN
345,304,SYR-2888,NaN,NAO ESPEC.,NaN,NaN,NaN
346,305,PPZ-1085,NaN,NAO ESPEC.,NaN,NaN,NaN
347,306,VPW-4905,NaN,NAO ESPEC.,NaN,NaN,NaN
348,307,JIO-1567,NaN,NAO ESPEC.,NaN,NaN,NaN
349,308,KVR-6935,NaN,NAO ESPEC.,NaN,NaN,NaN
